In [1]:
import numpy as np
import pandas as pd
import uavsar_pytools

In [2]:
from uavsar_pytools import UavsarCollection, UavsarImage

In [ ]:
col_name = 'Grand Mesa, CO'

## Working directory to save files into
work_d = '/Users/rdcrlzh1/Documents/uavsar-coherence/data/grmesa/uavsar'


## Optional dates to check between
dates = (f'2016-12-01',f'2018-06-01')

collection = UavsarCollection(collection = col_name, work_dir = work_d, dates = dates, clean = False, inc = True)

# Optional keywords: to keep binary files use `clean = False`, to download incidence angles 
# with each image use `inc = True`, for only certain pols use `pols = ['VV','HV']`.
# See docstring of class for full list.

collection.collection_to_tiffs()

# incidence angle files too

In [3]:
## Working directory to save files into
work_d = '/Users/rdcrlzh1/Documents/uavsar-coherence/data/grmesa/uavsar'

inc_urls = ['https://uavsar.asf.alaska.edu/UA_grmesa_08006_17043_006_170331_L090_CX_01/grmesa_08006_17043_006_170331_L090_CX_01.inc',
'https://uavsar.asf.alaska.edu/UA_GrMesa_08112_19047_005_190715_L090_CX_02/GrMesa_08112_19047_005_190715_L090_CX_02.inc',
'https://uavsar.asf.alaska.edu/UA_grmesa_26006_17043_005_170331_L090_CX_01/grmesa_26006_17043_005_170331_L090_CX_01.inc',
'https://uavsar.asf.alaska.edu/UA_grmesa_07805_17043_004_170331_L090_CX_01/grmesa_07805_17043_004_170331_L090_CX_01.inc']
# https://uavsar.asf.alaska.edu/UA_grmesa_27416_21021_005_210322_L090_CX_01/grmesa_27416_21021_005_210322_L090_CX_01.inc
# https://uavsar.asf.alaska.edu/UA_GrMesa_26108_19047_004_190715_L090_CX_02/GrMesa_26108_19047_004_190715_L090_CX_02.inc

for url in inc_urls:
    UavsarImage(url = url, work_dir= work_d, clean = False).url_to_tiff()

Searching with: hgt


Searching with: hgt


Searching with: hgt
Searching with: hgt


In [228]:
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rxa
from pathlib import Path
from shapely.geometry import box, polygon

In [233]:
from rasterio.enums import Resampling

aoi = box(*gpd.read_file('/Users/rdcrlzh1/Documents/uavsar-coherence/data/grmesa/aoi/aoi.shp')['geometry'].values[0].bounds)
aoi = box(-108.2671821858561, 38.96, -107.9429720033735, 39.109457285536415)
work_d = Path('/Users/rdcrlzh1/Documents/uavsar-coherence/data/grmesa/')
cors = []
for d in list(work_d.joinpath('uavsar').glob('*grd')):
    for f in d.glob('*cor.grd.tiff'):
        ann = pd.read_csv(next(d.glob('*grd.csv')), index_col= 0)
        t1, t2 = pd.to_datetime(ann['start time of acquisition for pass 1']['value']), pd.to_datetime(ann['start time of acquisition for pass 2']['value'])
        t = f'{t1}_{t2}'
        pol = f.stem.split('_')[-2][-2:]
        heading = f.stem.split('_')[1][:3]

        cor = rxa.open_rasterio(f).rio.clip_box(*aoi.bounds).squeeze('band', drop = True)
        # cor = cor.isel(x = slice(0, -1, 100), y = slice(0, -1, 100))
        cor = cor.rio.write_nodata(np.nan)
        
        if len(cors) != 0:            
            cor = cor.rio.reproject_match(cors[0], Resampling = Resampling.bilinear)

        cor = cor.expand_dims(times = [t]).expand_dims(pol = [pol])
        cor = cor.assign_coords(time1 = t1).assign_coords(time2 = t2).assign_coords(heading = heading)
        cors.append(cor)

ds = xr.combine_by_coords(cors)
ds.to_netcdf('/Users/rdcrlzh1/Documents/uavsar-coherence/data/grmesa/gm_cors.nc')